In [1]:
import pandas as pd
import xlwings as xw
df = pd.read_csv(r'C:\Users\chpar10\Downloads\euromillions.csv')
df.sample(5)

,No.,Day,DD,MMM,YYYY,N1,N2,N3,N4,N5,L1,L2,Jackpot,Wins
103,1149,Tue,25,Sep,2018,38,6,20,30,15,4,7,124411811,0
935,317,Fri,5,Mar,2010,19,18,49,12,43,9,3,41706216,1
283,969,Tue,3,Jan,2017,34,27,49,23,19,11,1,21591336,0
709,543,Tue,4,Dec,2012,29,5,28,44,10,2,4,34505412,0
468,784,Fri,27,Mar,2015,30,39,32,44,2,10,6,47673633,0


In [10]:
wb = xw.Book()

In [12]:
print(xw.apps)

Apps([<Excel App 76>])


In [14]:
#creates a worksheet object assigns it to ws
ws = wb.sheets["Sheet1"]
#checks that wb.sheets[0] equals ws
print(ws == wb.sheets[0])

True


In [16]:
ws.name = "EuroMillions"
ws.range("A1").value = df

In [20]:
ws.clear_contents()
ws.range("A1").options(index=False).value = df

In [21]:
last_row = ws.range(1,1).end('down').row
print("The last row is {row}.".format(row=last_row))
print("The DataFrame df has {rows} rows.".format(rows=df.shape[0]))

The last row is 1253.
The DataFrame df has 1252 rows.


In [22]:
ws.range(
"A2:N{row}".format(row=last_row)
).api.Sort(Key1=ws.range("A:A").api, Order1=1)

True

In [23]:
ws.range("O1").value = "Date"

In [24]:
ws.range("O2").value = "=C2&D2&RIGHT(E2, 2)"

In [25]:
ws.range("O2").api.AutoFill(
ws.range("O2:O{row}".format(row=last_row)).api,
0
)

True

In [26]:
from xlwings.constants import AutoFillType

ws.range("O2").api.AutoFill(
ws.range("O2:O{row}".format(row=last_row)).api,
    AutoFillType.xlFillDefault
)

True

In [29]:
for arg4 in (0, 1):
    for arg3 in (0,1):
        for arg2 in (0,1):
            for arg1 in (0,1):
                print(ws.range("O2").get_address(arg1, arg2, arg3, arg4))

O2
O$2
$O2
$O$2
EuroMillions!O2
EuroMillions!O$2
EuroMillions!$O2
EuroMillions!$O$2
[Book2]EuroMillions!O2
[Book2]EuroMillions!O$2
[Book2]EuroMillions!$O2
[Book2]EuroMillions!$O$2
[Book2]EuroMillions!O2
[Book2]EuroMillions!O$2
[Book2]EuroMillions!$O2
[Book2]EuroMillions!$O$2


In [31]:
def autofill(worksheet, cell, last_row):
    rg_cell = worksheet.range(cell)
    to_fill = "{col}{top_row}:{col}{last_row}".format(
      col=rg_cell.get_address(0,0)[0],
      top_row=rg_cell.row,
      last_row=last_row
    )
    rg_cell.api.Autofill(worksheet.range(to_fill).api, 0)

In [32]:
print(type(ws.range("O2:O{row}".format(row=last_row)).value))

<class 'list'>


In [33]:
print(ws.range('O2:O{row}'.format(row=last_row)).value[:10])

['13Feb04', '20Feb04', '27Feb04', '5Mar04', '12Mar04', '19Mar04', '26Mar04', '2Apr04', '9Apr04', '16Apr04']


In [34]:
ws.range('O2').options(transpose=True).value\
= ws.range('O2:O{row}'.format(row=last_row)).value
ws.range('C:E').api.Delete()

True

In [35]:
import datetime

for day in ws.range('L2:L{}'.format(last_row)):
# checks if day is not prior to the change of the rules
    if day.value <= datetime.datetime(2016, 9, 24, 0, 0):
        # since day is past the modification date,
        # the row we want is the previous one, hence minus 1
        to_delete = int(day.get_address(0, 0)[1:])-1
        # leave the for cycle
        break

In [36]:
ws.range('2:{}'.format(to_delete)).api.Delete()

True

In [38]:
ws.range('1:1').api.Font.Bold = True
#ws.range('J:J').number_format = "£##.##0,,' M'"
last_column = ws.range(1,1).end('right').get_address(0,0)[0]
ws.range('A1:{}1'.format(last_column)).api.Borders(9).LineStyle = -4119
ws.autofit()

In [39]:
ws.range('A:L').api.ColumnWidth = ws.range('J:J').api.ColumnWidth
wb.sheets.add('Frequencies')
frequencies = wb.sheets['Frequencies']

# add a header for the numbers
frequencies.range('A1').value = 'Number'
# populate the fifty cells immediately below with the numbers 1 through 50
# since we're starting from the second row, we'll want to substract 1 from the row
frequencies.range('A2:A51').value = '=ROW()-1'

In [40]:
# add a header for the frequencies
frequencies.range('B1').value = 'Frequency'
# insert on B2 the result of a standard Excel formula
frequencies.range('B2').value = '=COUNTIF(Euromillions!$C$2:$G$201,Frequencies!A2)'
autofill(frequencies, 'B2', 51)

In [41]:
frequencies.range('D1').value = 'Lucky Star'
frequencies.range('E1').value = 'Frequency'
frequencies.range('D2:D13').value = '=ROW()-1'
frequencies.range('E2').value =\
'=COUNTIF(EuroMillions!$H$2:$I$201,Frequencies!D2)'
autofill(frequencies, 'E2', 13)
frequencies.autofit()

In [42]:
wb.sheets.add('Graphs')
graphs = wb.sheets['Graphs']

In [43]:
nr_freq = xw.Chart()


In [44]:
nr_freq.name = 'Number Frequencies'
nr_freq.set_source_data(frequencies.range('Frequencies!B1:B51'))


In [45]:
nr_freq.api[1].FullSeriesCollection(1).XValues = '=Frequencies!A2:A51'


In [46]:
nr_freq.chart_type = 'column_clustered'

In [47]:
nr_freq.height = 250
nr_freq.width = 750

In [48]:
nr_freq.api[1].SetElement(2)  # Place chart title at the top
nr_freq.api[1].ChartTitle.Text = 'Number Frequencies'

In [49]:
nr_freq.api[1].HasLegend = 0

In [50]:
nr_freq.api[1].Axes(1).TickLabelSpacing = 1

In [51]:
graphs.shapes.api('Number Frequencies').Line.Visible = 0

In [52]:
ls_freq = xw.Chart()
ls_freq.top = 250
ls_freq.name = 'Lucky Star Frequencies'
ls_freq.set_source_data(frequencies.range('Frequencies!E1:E13'))
ls_freq.api[1].FullSeriesCollection(1).XValues = '=Frequencies!D2:D13'
ls_freq.chart_type = 'column_clustered'
ls_freq.height = 250
ls_freq.width = 750
ls_freq.api[1].SetElement(2)
ls_freq.api[1].ChartTitle.Text = 'Lucky Star Frequencies'
ls_freq.api[1].HasLegend = 0
ls_freq.api[1].Axes(1).TickLabelSpacing = 1
graphs.shapes.api('Lucky Star Frequencies').Line.Visible = 0

In [53]:
jackpot = xw.Chart()
jackpot.top = 500
jackpot.name = 'Jackpot'
last_row = ws.range(1,1).end('down').row
jackpot.set_source_data(ws.range('Euromillions!J2:J{}'.format(last_row)))
jackpot.api[1].FullSeriesCollection(1).XValues\
= 'Euromillions!L2:L{}'.format(last_row)
jackpot.chart_type = 'line'
jackpot.height = 250
jackpot.width = 750
jackpot.api[1].SetElement(2)
jackpot.api[1].ChartTitle.Text = 'Jackpot'
jackpot.api[1].HasLegend = 0
graphs.shapes.api('Jackpot').Line.Visible = 0

ref to
https://www.dataquest.io/blog/python-excel-xlwings-tutorial/